In [ ]:
!nvidia-smi

Thu Aug 19 18:49:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import numpy as np
import os
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.models import model_from_json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%time
import zipfile
import os

zip_ref = zipfile.ZipFile('/content/drive/MyDrive/ISIC_2019_Training_Input.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()

CPU times: user 1min 12s, sys: 22.7 s, total: 1min 35s
Wall time: 7min 48s


In [ ]:
len(os.listdir('/tmp/ISIC_2019_Training_Input/'))

25333

In [ ]:
import pandas as pd
data_set=pd.read_csv("ISIC_2019_Training_GroundTruth.csv")

In [ ]:
data_set.head()

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK
0,ISIC_0000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0000001,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0000002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0000003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0000004,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
y = data_set[['MEL','NV','BCC','AK','BKL','DF','VASC','SCC']]
y.head()

,MEL,NV,BCC,AK,BKL,DF,VASC,SCC
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import numpy as np
y_arr = y.to_numpy()
y_arr

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
y_train = np.where(y_arr)[1]

In [ ]:
y_train = pd.Series(y_train,name='label')

In [ ]:
new_df = pd.concat([data_set['image'],y_train],axis=1)
new_df.iloc[24000:25000]

,image,label
24000,ISIC_0071150,2
24001,ISIC_0071151,7
24002,ISIC_0071153,1
24003,ISIC_0071155,6
24004,ISIC_0071156,2
...,...,...
24995,ISIC_0072741,3
24996,ISIC_0072742,1
24997,ISIC_0072744,2
24998,ISIC_0072745,1


In [ ]:
new_df.to_csv('../img_data.csv',index=False)

In [ ]:
import shutil, os
import pandas as pd

labels = pd.read_csv("../img_data.csv")
#print(labels)
labels = labels.sort_values('label')
print(labels)
class_names = list(labels.label.unique())

              image  label
12665  ISIC_0034068      0
4213   ISIC_0025616      0
19063  ISIC_0063302      0
12808  ISIC_0034211      0
4208   ISIC_0025611      0
...             ...    ...
17774  ISIC_0061243      7
15646  ISIC_0057890      7
19852  ISIC_0064593      7
22239  ISIC_0068367      7
22132  ISIC_0068203      7

[25331 rows x 2 columns]


In [ ]:
for i in class_names:
  os.makedirs(os.path.join('train_', str(i)))

for c in class_names: # Category Name
  for i in list(labels[labels['label']==c]['image']): # Image Id
    get_image = os.path.join('/tmp/ISIC_2019_Training_Input/', str(i)+".jpg") # Path to Images
    move_image_to_cat = shutil.move(get_image, 'train_/'+str(c))

In [ ]:
img_height = 150
img_width = 150
batch_size = 32

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150,150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(8))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'train_',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'train_',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

Found 25331 images belonging to 8 classes.
Found 25331 images belonging to 8 classes.


In [ ]:
total_sample = train_generator.n

In [ ]:
%%time
model.fit_generator(
        train_generator, 
        steps_per_epoch=int(total_sample/batch_size),  
        epochs=10,
        verbose=1)


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
791/791 [==============================] - 674s 815ms/step - loss: 1.3350 - accuracy: 0.5164
Epoch 2/10
791/791 [==============================] - 633s 800ms/step - loss: 1.2592 - accuracy: 0.5464
Epoch 3/10
791/791 [==============================] - 635s 803ms/step - loss: 1.2125 - accuracy: 0.5650
Epoch 4/10
791/791 [==============================] - 646s 817ms/step - loss: 1.1688 - accuracy: 0.5831
Epoch 5/10
791/791 [==============================] - 640s 809ms/step - loss: 1.1394 - accuracy: 0.5945
Epoch 6/10
791/791 [==============================] - 629s 795ms/step - loss: 1.1170 - accuracy: 0.6041
Epoch 7/10
791/791 [==============================] - 627s 793ms/step - loss: 1.0998 - accuracy: 0.6035
Epoch 8/10
791/791 [==============================] - 625s 790ms/step - loss: 1.0913 - accuracy: 0.6050
Epoch 9/10
791/791 [==============================] - 634s 801ms/step - loss: 1.0816 - accuracy: 0.6102
Epoch 10/10
791/791 [==============================] - 647s 818m

In [ ]:
%%time
scores = model.evaluate(train_generator, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

accuracy: 63.25%


In [ ]:
 !pip install h5py

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
%%time
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#score = loaded_model.evaluate(train_generator, verbose=0)
#print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk
CPU times: user 459 ms, sys: 390 ms, total: 849 ms
Wall time: 5.98 s


In [ ]:
from PIL import Image
image = Image.open('ISIC_0000014.jpg')
print(image.size)

resized_image = image.resize((150,150))
print(resized_image.size)


(1022, 767)
(150, 150)


In [ ]:
img1 = img_to_array(resized_image)
type(img1)
pixels = img1.astype('float32')
pixels /= 255.0
print(pixels.shape)
pixels = pixels.reshape(1,150,150,3)
pixels.shape
#img1.shape
#img1.reshape(1,a)
loaded_model.predict(pixels)

(150, 150, 3)


array([[0.81779355, 0.9466005 , 0.38100457, 0.12943362, 0.7303076 ,
        0.05171845, 0.02138169, 0.10217568]], dtype=float32)

In [ ]:
pixels = np.expand_dims(pixels, axis=0)
pixels.shape
#loaded_model.predict(pixels)

(1, 150, 150, 3)